# mmmeld Go Implementation - Validation Tests

This notebook contains parameterized validation tests for the mmmeld Go binary. We'll test various scenarios and rules using different combinations of media sources.

**Note:** This notebook uses Python as a test harness to call the compiled Go binary and visualize results.

In [ ]:
import subprocess
import os
from IPython.display import Video

def run_mmmeld(params):
    output_file = f"test_output_{params[params.index('--output') + 1]}"
    # Use the Go binary - 'mmmeld' should be in PATH (via bin/)
    cmd = ["mmmeld", "--autofill", "--cleanup"] + params + ["--output", output_file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(" ".join(cmd)) 
    print(result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)
    return result.returncode == 0, output_file

def display_output(output_file):
    if os.path.exists(output_file):
        return Video(output_file)
    else:
        return f"Output file {output_file} not found."

# Define media sources
local_audio_short = '../test_media/10_seconds.wav'  # 10 seconds
local_audio_long = '../test_media/30_seconds.mp3'  # 30 seconds
youtube_audio = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'  # Rick Astley - Never Gonna Give You Up
local_image1 = '../test_media/square.jpg' # square
local_image2 = '../test_media/16_9.png' # 16:9
local_video_short = '../test_media/5_seconds.mp4'  # 5 seconds
local_video_short2 = '../test_media/5_seconds_2.mp4'  # 5 seconds
local_video_med = '../test_media/15_seconds.mp4'  # 15 seconds
local_video_med2 = '../test_media/15_seconds2.mp4'  # 15 seconds
local_video_long = '../test_media/3_minutes.webm'  # 3 minutes
youtube_video = 'https://www.youtube.com/watch?v=9bZkp7q19f0'  # PSY - GANGNAM STYLE
bg_music = '../test_media/IndigoMirage.mp3'

# Text for text-to-speech generation
tts_text = "To be, or not to be, that is the question."

## Test 1: Single Image with Main Audio

Expected behavior: The image should be displayed for the entire duration of the audio.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--output", "test1.mp4"])
print(f"Test 1 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 2: Single Image without Main Audio

Expected behavior: The image should be displayed for 5 seconds.

In [ ]:
success, output_file = run_mmmeld(["--image", local_image1, "--output", "test2.mp4"])
print(f"Test 2 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 3: Single Video with Main Audio (Video Shorter)

Expected behavior: The video should loop to match the audio duration.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", local_video_short, "--output", "test3.mp4"])
print(f"Test 3 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 4: Single Video with Main Audio (Video Longer)

Expected behavior: The video should be cut to match the audio duration.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_video_med, "--output", "test4.mp4"])
print(f"Test 4 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 5: Multiple Images with Main Audio

Expected behavior: Images should split the time equally throughout the audio duration.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_image1},{local_image2}", "--output", "test5.mp4"])
print(f"Test 5 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 6: Image and Video with Main Audio

Expected behavior: Image followed by video, with timing based on audio duration.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", f"{local_image1},{local_video_short}", "--output", "test6.mp4"])
print(f"Test 6 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 7: With Background Music

Expected behavior: Background music should loop and fade out at the end.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--bg-music", bg_music, "--output", "test7.mp4"])
print(f"Test 7 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 8: Custom Audio Margins

Expected behavior: Video should have 1 second lead-in and 3 second tail with fade-out.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--audiomargin", "1.0,3.0", "--output", "test8.mp4"])
print(f"Test 8 {'passed' if success else 'failed'}")
display_output(output_file)